CHANGELOG

v0.0 6/16/23:
- Dice score

v0.1 6/19/23:
- added in pearson correlation

v0.2 6/19/23:
- added hausdorff distance
- added IoU
- added AUC ROC
- added AUC PR

v0.3 6/24/23
- added in access to jerry_losses.py

In [ ]:
from jarvis.utils.general import gpus
gpus.autoselect()

import numpy as np, pandas as pd, tensorflow as tf, os
from tensorflow.keras import Input, Model,layers
from pathlib import Path
from jarvis.train import models, custom
from jarvis.utils.display import imshow
from scipy.spatial.distance import directed_hausdorff
from IPython.display import clear_output, HTML, Javascript, display

import sys  
sys.path.append('/home/jjlou/Jerry/jerry_packages')
from jerry_utils import restart_kernel, load_dataset
import jerry_losses, jerry_metrics 

[ 2024-01-09 11:27:53 ] CUDA_VISIBLE_DEVICES automatically set to: 1           


In [ ]:
name = 'Attention_Unfrozen_EfficientNetV2L_data_v0.0.py'
root = '/home/jjlou/Jerry/wsi-arterio/arteriosclerotic_vessel_detection_and_fine_segmentation/Vessel_WallsLumen_Segmentation/data'

custom_objects = {
    'dice_all': jerry_metrics.dice_metric(cls=1),
    'hausdorff_all': jerry_metrics.hausdorff_metric(cls=1),
    'focal_dice_like_loss_multiclass_weighted': jerry_losses.focal_dice_like_loss_multiclass_weighted
}

In [ ]:
Valid0 = ['D-436','D-870', 'D-343']
Valid1 = ['D-297', 'D-916', 'UCI-15-12']
Valid2 = ['D-322', 'D-562']

Valid = [Valid0, Valid1, Valid2]

In [ ]:
df_model_vl = pd.DataFrame(index=range(len(Valid)))
df_model_vl['Train mean'] = ''
df_model_vl['Train std'] = ''
df_model_vl['Train min'] = ''
df_model_vl['Train 25%'] = ''
df_model_vl['Train median'] = ''
df_model_vl['Train 75%'] = ''
df_model_vl['Train max'] = ''
df_model_vl['Train pearson mean'] = ''
df_model_vl['Train p mean'] = ''
df_model_vl['Train hausdorff mean'] = ''
df_model_vl['Train mIoU mean'] = ''
df_model_vl['Train AUC-ROC mean'] = ''
df_model_vl['Train AUC-PR mean'] = ''
df_model_vl['Valid mean'] = ''
df_model_vl['Valid std'] = ''
df_model_vl['Valid min'] = ''
df_model_vl['Valid 25%'] = ''
df_model_vl['Valid median'] = ''
df_model_vl['Valid 75%'] = ''
df_model_vl['Valid max'] = ''
df_model_vl['Valid pearson mean'] = ''
df_model_vl['Valid p mean'] = ''
df_model_vl['Valid pearson mean'] = ''
df_model_vl['Valid p mean'] = ''
df_model_vl['Valid hausdorff mean'] = ''
df_model_vl['Valid mIoU mean'] = ''
df_model_vl['Valid AUC-ROC mean'] = ''
df_model_vl['Valid AUC-PR mean'] = ''
    
df_model_v = pd.DataFrame(index=range(len(Valid)))
df_model_v['Train mean'] = ''
df_model_v['Train std'] = ''
df_model_v['Train min'] = ''
df_model_v['Train 25%'] = ''
df_model_v['Train median'] = ''
df_model_v['Train 75%'] = ''
df_model_v['Train max'] = ''
df_model_v['Train pearson mean'] = ''
df_model_v['Train p mean'] = ''
df_model_v['Train hausdorff mean'] = ''
df_model_v['Train mIoU mean'] = ''
df_model_v['Train AUC-ROC mean'] = ''
df_model_v['Train AUC-PR mean'] = ''
df_model_v['Valid mean'] = ''
df_model_v['Valid std'] = ''
df_model_v['Valid min'] = ''
df_model_v['Valid 25%'] = ''
df_model_v['Valid median'] = ''
df_model_v['Valid 75%'] = ''
df_model_v['Valid max'] = ''
df_model_v['Valid pearson mean'] = ''
df_model_v['Valid p mean'] = ''
df_model_v['Valid pearson mean'] = ''
df_model_v['Valid p mean'] = ''
df_model_v['Valid pearson mean'] = ''
df_model_v['Valid p mean'] = ''
df_model_v['Valid hausdorff mean'] = ''
df_model_v['Valid mIoU mean'] = ''
df_model_v['Valid AUC-ROC mean'] = ''
df_model_v['Valid AUC-PR mean'] = ''
    
df_model_l = pd.DataFrame(index=range(len(Valid)))
df_model_l['Train mean'] = ''
df_model_l['Train std'] = ''
df_model_l['Train min'] = ''
df_model_l['Train 25%'] = ''
df_model_l['Train median'] = ''
df_model_l['Train 75%'] = ''
df_model_l['Train max'] = ''
df_model_l['Train pearson mean'] = ''
df_model_l['Train p mean'] = ''
df_model_l['Train hausdorff mean'] = ''
df_model_l['Train mIoU mean'] = ''
df_model_l['Train AUC-ROC mean'] = ''
df_model_l['Train AUC-PR mean'] = ''
df_model_l['Valid mean'] = ''
df_model_l['Valid std'] = ''
df_model_l['Valid min'] = ''
df_model_l['Valid 25%'] = ''
df_model_l['Valid median'] = ''
df_model_l['Valid 75%'] = ''
df_model_l['Valid max'] = ''
df_model_l['Valid pearson mean'] = ''
df_model_l['Valid p mean'] = ''
df_model_l['Valid pearson mean'] = ''
df_model_l['Valid p mean'] = ''
df_model_l['Valid pearson mean'] = ''
df_model_l['Valid p mean'] = ''
df_model_l['Valid hausdorff mean'] = ''
df_model_l['Valid mIoU mean'] = ''
df_model_l['Valid AUC-ROC mean'] = ''
df_model_l['Valid AUC-PR mean'] = ''

In [ ]:
for v in Valid:
    v_path = f"{root}/K{Valid.index(v)}"
    model_path = f'{v_path}/models/{name}.hdf5'
    load_path = f'{v_path}/models/{name}_K{Valid.index(v)}'
    save_path = f'{root}/models_results'
    
    df_train_vl = pd.read_csv(f'{load_path}_train_vl.csv')
    df_valid_vl = pd.read_csv(f'{load_path}_valid_vl.csv')
    df_train_v = pd.read_csv(f'{load_path}_train_v.csv')
    df_valid_v = pd.read_csv(f'{load_path}_valid_v.csv')
    df_train_l = pd.read_csv(f'{load_path}_train_l.csv')
    df_valid_l = pd.read_csv(f'{load_path}_valid_l.csv')
    
    i = Valid.index(v)
    
    #Vessel and Lumen
    #train
    df_model_vl['Train mean'][i] = df_train_vl['Train dice score'].mean()
    df_model_vl['Train std'][i] = df_train_vl['Train dice score'].std()
    df_model_vl['Train min'][i] = df_train_vl['Train dice score'].min()
    df_model_vl['Train 25%'][i] = df_train_vl['Train dice score'].quantile(q=0.25)
    df_model_vl['Train median'][i] = df_train_vl['Train dice score'].median()
    df_model_vl['Train 75%'][i] = df_train_vl['Train dice score'].quantile(q=0.75)
    df_model_vl['Train max'][i] = df_train_vl['Train dice score'].max()
    df_model_vl['Train pearson mean'][i] = df_train_vl['Train pearson'].mean()
    df_model_vl['Train p mean'][i] = df_train_vl['Train p'].mean()
    df_model_vl['Train hausdorff mean'][i] = df_train_vl['Train hausdorff'].mean()
    df_model_vl['Train mIoU mean'][i] = df_train_vl['Train mIoU'].mean()
    df_model_vl['Train AUC-ROC mean'][i] = df_train_vl['Train ROC'].mean()
    df_model_vl['Train AUC-PR mean'][i] = df_train_vl['Train PR'].mean()

    #valid
    df_model_vl['Valid mean'][i] = df_valid_vl['Valid dice score'].mean()
    df_model_vl['Valid std'][i] = df_valid_vl['Valid dice score'].std()
    df_model_vl['Valid min'][i] = df_valid_vl['Valid dice score'].min()
    df_model_vl['Valid 25%'][i] = df_valid_vl['Valid dice score'].quantile(q=0.25)
    df_model_vl['Valid median'][i] = df_valid_vl['Valid dice score'].median()
    df_model_vl['Valid 75%'][i] = df_valid_vl['Valid dice score'].quantile(q=0.75)
    df_model_vl['Valid max'][i] = df_valid_vl['Valid dice score'].max()
    df_model_vl['Valid pearson mean'][i] = df_valid_vl['Valid pearson'].mean()
    df_model_vl['Valid p mean'][i] = df_valid_vl['Valid p'].mean()
    df_model_vl['Valid hausdorff mean'][i] = df_valid_vl['Valid hausdorff'].mean()
    df_model_vl['Valid mIoU mean'][i] = df_valid_vl['Valid mIoU'].mean()
    df_model_vl['Valid AUC-ROC mean'][i] = df_valid_vl['Valid ROC'].mean()
    df_model_vl['Valid AUC-PR mean'][i] = df_valid_vl['Valid PR'].mean()
    
    #Vessel
    #train
    df_model_v['Train mean'][i] = df_train_v['Train dice score'].mean()
    df_model_v['Train std'][i] = df_train_v['Train dice score'].std()
    df_model_v['Train min'][i] = df_train_v['Train dice score'].min()
    df_model_v['Train 25%'][i] = df_train_v['Train dice score'].quantile(q=0.25)
    df_model_v['Train median'][i] = df_train_v['Train dice score'].median()
    df_model_v['Train 75%'][i] = df_train_v['Train dice score'].quantile(q=0.75)
    df_model_v['Train max'][i] = df_train_v['Train dice score'].max()
    df_model_v['Train pearson mean'][i] = df_train_v['Train pearson'].mean()
    df_model_v['Train p mean'][i] = df_train_v['Train p'].mean()
    df_model_v['Train hausdorff mean'][i] = df_train_v['Train hausdorff'].mean()
    df_model_v['Train mIoU mean'][i] = df_train_v['Train mIoU'].mean()
    df_model_v['Train AUC-ROC mean'][i] = df_train_v['Train ROC'].mean()
    df_model_v['Train AUC-PR mean'][i] = df_train_v['Train PR'].mean()

    #valid
    df_model_v['Valid mean'][i] = df_valid_v['Valid dice score'].mean()
    df_model_v['Valid std'][i] = df_valid_v['Valid dice score'].std()
    df_model_v['Valid min'][i] = df_valid_v['Valid dice score'].min()
    df_model_v['Valid 25%'][i] = df_valid_v['Valid dice score'].quantile(q=0.25)
    df_model_v['Valid median'][i] = df_valid_v['Valid dice score'].median()
    df_model_v['Valid 75%'][i] = df_valid_v['Valid dice score'].quantile(q=0.75)
    df_model_v['Valid max'][i] = df_valid_v['Valid dice score'].max()
    df_model_v['Valid pearson mean'][i] = df_valid_v['Valid pearson'].mean()
    df_model_v['Valid p mean'][i] = df_valid_v['Valid p'].mean()
    df_model_v['Valid hausdorff mean'][i] = df_valid_v['Valid hausdorff'].mean()
    df_model_v['Valid mIoU mean'][i] = df_valid_v['Valid mIoU'].mean()
    df_model_v['Valid AUC-ROC mean'][i] = df_valid_v['Valid ROC'].mean()
    df_model_v['Valid AUC-PR mean'][i] = df_valid_v['Valid PR'].mean()
    
    #Lumen
    #train
    df_model_l['Train mean'][i] = df_train_l['Train dice score'].mean()
    df_model_l['Train std'][i] = df_train_l['Train dice score'].std()
    df_model_l['Train min'][i] = df_train_l['Train dice score'].min()
    df_model_l['Train 25%'][i] = df_train_l['Train dice score'].quantile(q=0.25)
    df_model_l['Train median'][i] = df_train_l['Train dice score'].median()
    df_model_l['Train 75%'][i] = df_train_l['Train dice score'].quantile(q=0.75)
    df_model_l['Train max'][i] = df_train_l['Train dice score'].max()
    df_model_l['Train pearson mean'][i] = df_train_l['Train pearson'].mean()
    df_model_l['Train p mean'][i] = df_train_l['Train p'].mean()
    df_model_l['Train hausdorff mean'][i] = df_train_l['Train hausdorff'].mean()
    df_model_l['Train mIoU mean'][i] = df_train_l['Train mIoU'].mean()
    df_model_l['Train AUC-ROC mean'][i] = df_train_l['Train ROC'].mean()
    df_model_l['Train AUC-PR mean'][i] = df_train_l['Train PR'].mean()

    #valid
    df_model_l['Valid mean'][i] = df_valid_l['Valid dice score'].mean()
    df_model_l['Valid std'][i] = df_valid_l['Valid dice score'].std()
    df_model_l['Valid min'][i] = df_valid_l['Valid dice score'].min()
    df_model_l['Valid 25%'][i] = df_valid_l['Valid dice score'].quantile(q=0.25)
    df_model_l['Valid median'][i] = df_valid_l['Valid dice score'].median()
    df_model_l['Valid 75%'][i] = df_valid_l['Valid dice score'].quantile(q=0.75)
    df_model_l['Valid max'][i] = df_valid_l['Valid dice score'].max()
    df_model_l['Valid pearson mean'][i] = df_valid_l['Valid pearson'].mean()
    df_model_l['Valid p mean'][i] = df_valid_l['Valid p'].mean()
    df_model_l['Valid hausdorff mean'][i] = df_valid_l['Valid hausdorff'].mean()
    df_model_l['Valid mIoU mean'][i] = df_valid_l['Valid mIoU'].mean()
    df_model_l['Valid AUC-ROC mean'][i] = df_valid_l['Valid ROC'].mean()
    df_model_l['Valid AUC-PR mean'][i] = df_valid_l['Valid PR'].mean()

In [ ]:
# load model
model = tf.keras.models.load_model(model_path, custom_objects=custom_objects)

df_model_vl['Trainable Params'] = np.sum([np.prod(v.get_shape()) for v in model.trainable_weights])
df_model_v['Trainable Params'] = np.sum([np.prod(v.get_shape()) for v in model.trainable_weights])
df_model_l['Trainable Params'] = np.sum([np.prod(v.get_shape()) for v in model.trainable_weights])
    
df_model_vl.to_csv(f'{save_path}/{name}_Color.BasicMorph.Aug_vl.csv')
df_model_v.to_csv(f'{save_path}/{name}_Color.BasicMorph.Aug_v.csv')
df_model_l.to_csv(f'{save_path}/{name}_Color.BasicMorph.Aug_l.csv')